📦 1. Imports and Setup

In [1]:
# Import necessary libraries
from sqlalchemy import Column, Integer, String, ForeignKey, Date, JSON, create_engine
from sqlalchemy.orm import relationship, declarative_base, sessionmaker
from pydantic import BaseModel, field_serializer
from typing import List
from datetime import date
from deepdiff import DeepDiff, Delta
from pprint import pprint

# Initialize SQLAlchemy Base
Base = declarative_base()

🚗 2. Define SQLAlchemy Models


In [2]:
# Define Car model
class Car(Base):
    __tablename__ = 'cars'
    id = Column(Integer, primary_key=True)
    license_plate = Column(String, unique=True, nullable=False)
    model = Column(String, nullable=False)
    
    # 1:N relationships
    service_records = relationship('ServiceRecord', back_populates='car')
    car_historic = relationship('CarHistoric', back_populates='car')


# Define Collaborator model
class Collaborator(Base):
    __tablename__ = 'collaborators'
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    
    # 1:N relationship
    service_records = relationship('ServiceRecord', back_populates='service_manager')


# Define ServiceRecord model
class ServiceRecord(Base):
    __tablename__ = 'service_records'
    id = Column(Integer, primary_key=True)
    service_date = Column(Date)
    description = Column(String)
    car_id = Column(Integer, ForeignKey('cars.id'))
    service_manager_id = Column(Integer, ForeignKey('collaborators.id'))
    
    car = relationship('Car', back_populates='service_records')
    service_manager = relationship('Collaborator', back_populates='service_records')


# Define CarHistoric model for tracking changes
class CarHistoric(Base):
    __tablename__ = 'car_change_logs'
    id = Column(Integer, primary_key=True)
    change_date = Column(Date)
    changes = Column(JSON)
    car_id = Column(Integer, ForeignKey('cars.id'))
    
    car = relationship('Car', back_populates='car_historic')


🏗️ 3. Define Pydantic Schemas

In [3]:
# Collaborator Pydantic schema
class CollaboratorSchema(BaseModel):
    id: int
    name: str
    
    class Config:
        from_attributes = True


# ServiceRecord Pydantic schema with custom date serializer
class ServiceRecordSchema(BaseModel):
    id: int
    service_date: date
    description: str
    service_manager: CollaboratorSchema

    @field_serializer("service_date")
    def serialize_service_date(self, service_date: date) -> str:
        return service_date.isoformat()
    
    class Config:
        from_attributes = True


# Car Pydantic schema with helper to convert from schema to SQLAlchemy object
class CarSchema(BaseModel):
    id: int
    license_plate: str
    model: str
    service_records: List[ServiceRecordSchema] = []
    
    def car_from_schema(car_schema) -> Car:
        car_instance = Car(
            id=car_schema['id'],
            license_plate=car_schema['license_plate'],
            model=car_schema['model']
        )
        for sr_data in car_schema['service_records']:
            sm_data = sr_data['service_manager']
            service_manager = Collaborator(id=sm_data['id'], name=sm_data['name'])
            service_record = ServiceRecord(
                id=sr_data['id'],
                service_date=date.fromisoformat(sr_data['service_date']),
                description=sr_data['description'],
                service_manager=service_manager
            )
            car_instance.service_records.append(service_record)
        return car_instance

    class Config:
        from_attributes = True


💾 4. Database Setup

In [4]:
# SQLite database setup
DATABASE_URL = "sqlite:///./database.db"
engine = create_engine(DATABASE_URL, connect_args={"check_same_thread": False})
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# Reset and create tables
Base.metadata.drop_all(bind=engine)
Base.metadata.create_all(bind=engine)


🚀 5. Insert Initial Data

In [5]:
# Initialize DB session
db = SessionLocal()

# Create sample Car and Collaborator
new_car = Car(id=1, license_plate="ABC123", model="Toyota Corolla")
collaborator_1 = Collaborator(id=1, name="John Doe")

db.add_all([new_car, collaborator_1])
db.commit()


🔧 6. Add Service Record

In [6]:
# Add a new service record for the car
new_service_record = ServiceRecord(
    id=1,
    service_date=date(2025, 5, 5),
    description="Oil change",
    car=new_car,
    service_manager=collaborator_1
)

db.add(new_service_record)
db.commit()
db.refresh(new_service_record)


📦 7. Serialize and Capture Initial State

In [7]:
# Serialize Car to schema and dump to dict
car_schema_v1 = CarSchema.model_validate(new_car)
car_dump_v1 = car_schema_v1.model_dump()
print(f'First version:\n{car_dump_v1}')


First version:
{'id': 1, 'license_plate': 'ABC123', 'model': 'Toyota Corolla', 'service_records': [{'id': 1, 'service_date': '2025-05-05', 'description': 'Oil change', 'service_manager': {'id': 1, 'name': 'John Doe'}}]}


✏️ 8. Modify Data and Capture Changes


In [8]:
# Update service date
new_service_record.service_date = date(2027, 7, 7)
db.commit()

# Serialize updated state
car_schema_v2 = CarSchema.model_validate(new_car)
car_dump_v2 = car_schema_v2.model_dump()
print(f'Updated version:\n{car_dump_v2}')


Updated version:
{'id': 1, 'license_plate': 'ABC123', 'model': 'Toyota Corolla', 'service_records': [{'id': 1, 'service_date': '2027-07-07', 'description': 'Oil change', 'service_manager': {'id': 1, 'name': 'John Doe'}}]}


🔍 9. Compute Differences

In [9]:


# Calculate differences between versions
diff = DeepDiff(car_dump_v1, car_dump_v2, ignore_order=True, report_repetition=True)
print(f'Differences detected:\n{diff}')

Differences detected:
{'values_changed': {"root['service_records'][0]['service_date']": {'new_value': '2027-07-07', 'old_value': '2025-05-05'}}}


📝 10. Log Historic Changes


In [10]:
# Log differences in CarHistoric
car_historic = CarHistoric(
    change_date=date.today(),
    changes=diff,
    car=new_car
)
db.add(car_historic)
db.commit()

⏪ 11. Rollback Changes

In [11]:
# Compute delta and rollback changes
delta = Delta(diff, bidirectional=True)
rollback_changes = car_dump_v2 - delta
print(f'Rollback state:\n{rollback_changes}')

# Convert rolled-back schema to Car object and update DB
car_rollback = CarSchema.car_from_schema(rollback_changes)
db.merge(car_rollback)
db.commit()
print(f'Successfully rolled back to:\n{car_dump_v1}')

Rollback state:
{'id': 1, 'license_plate': 'ABC123', 'model': 'Toyota Corolla', 'service_records': [{'id': 1, 'service_date': '2025-05-05', 'description': 'Oil change', 'service_manager': {'id': 1, 'name': 'John Doe'}}]}
Successfully rolled back to:
{'id': 1, 'license_plate': 'ABC123', 'model': 'Toyota Corolla', 'service_records': [{'id': 1, 'service_date': '2025-05-05', 'description': 'Oil change', 'service_manager': {'id': 1, 'name': 'John Doe'}}]}
